In [1]:
!pip install mesa


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 13.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.2/418.2 kB 10.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━

In [2]:
!pip install --upgrade mesa


In [5]:
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random
import math

class EnvironmentPatch(Agent):
    """An environment patch that holds a fitness value."""
    def __init__(self, unique_id, model, val=0):
        super().__init__(unique_id, model)
        self.val = val  # Fitness value

class RobotAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.vx = random.normalvariate(0, 1)
        self.vy = random.normalvariate(0, 1)
        self.personal_best_val = -float('inf')
        self.personal_best_x = None
        self.personal_best_y = None

def update_personal_best(self):
    # Get agents or objects in the current grid cell
    cell_contents = self.model.grid.get_cell_list_contents([self.pos])
    # Assuming the first object in the cell is what we're interested in; adjust as necessary
    if cell_contents:
        current_val = cell_contents[0].val
        if current_val > self.personal_best_val:
            self.personal_best_val = current_val
            self.personal_best_x, self.personal_best_y = self.pos
    else:
        # Handle the case where cell_contents might be empty
        print("Warning: Attempted to access an empty or non-existent cell.")


    def move(self):
        new_x = int((self.pos[0] + self.vx) % self.model.grid.width)
        new_y = int((self.pos[1] + self.vy) % self.model.grid.height)
        self.model.grid.move_agent(self, (new_x, new_y))

    def step(self):
        self.move()
        self.update_personal_best()
        # PSO velocity update based on personal and global best
        inertia_weight = 0.5
        cognitive_constant = 0.3
        social_constant = 0.3
        r1, r2 = random.random(), random.random()

        self.vx = (inertia_weight * self.vx +
                   cognitive_constant * r1 * (self.personal_best_x - self.pos[0]) +
                   social_constant * r2 * (self.model.global_best_x - self.pos[0]))
        self.vy = (inertia_weight * self.vy +
                   cognitive_constant * r1 * (self.personal_best_y - self.pos[1]) +
                   social_constant * r2 * (self.model.global_best_y - self.pos[1]))

class PSOModel(Model):
    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.current_id = 0

        # Initialize global best values
        self.global_best_val = -float('inf')
        self.global_best_x = None
        self.global_best_y = None

        # Initialize environment patches correctly
        for (x, y) in [(x, y) for x in range(width) for y in range(height)]:
            unique_id = self.next_id()
            val = random.uniform(0, 1)  # Assuming a random value for demonstration
            patch = EnvironmentPatch(unique_id, self, val=val)
            self.grid.place_agent(patch, (x, y))

        # Initialize agents
        for i in range(self.num_agents):
            unique_id = self.next_id()
            agent = RobotAgent(unique_id, self)
            x, y = (random.randrange(self.grid.width), random.randrange(self.grid.height))
            self.grid.place_agent(agent, (x, y))
            self.schedule.add(agent)


    def step(self):
        self.schedule.step()
        # Update global best based on agents' personal bests
        for agent in self.schedule.agents:
            if isinstance(agent, RobotAgent) and agent.personal_best_val > self.global_best_val:
                self.global_best_val = agent.personal_best_val
                self.global_best_x, self.global_best_y = agent.personal_best_x, agent.personal_best_y
        self.schedule.step()

# Example of initializing and running the model
model = PSOModel(100, 50, 50)
for i in range(100):
    model.step()

In [6]:
def agent_portrayal(agent):
    if isinstance(agent, RobotAgent):
        portrayal = {"Shape": "circle", "Color": "blue", "Filled": "true", "Layer": 1, "r": 0.5}
    elif isinstance(agent, EnvironmentPatch):
        # Assuming 'val' ranges from 0 to 1, adjust colors as needed
        color_intensity = int(agent.val * 255)
        portrayal = {"Shape": "rect", "w": 1, "h": 1, "Color": f"rgb(0,{color_intensity},0)", "Filled": "true", "Layer": 0}
    return portrayal


In [11]:
# from mesa.visualization.ModularVisualization import ModularServer
# from mesa.visualization.modules import CanvasGrid
# # from mesa.visualization.UserParam import UserSettableParameter
# # from mesa.visualization.parameters import UserSettableParameter
# from mesa import UserSettableParameter


# def model_params():
#     return {"N": UserSettableParameter('slider', 'Number of Robots', value=100, min_value=10, max_value=200, step=10),
#             "width": 50,
#             "height": 50}

# grid = CanvasGrid(agent_portrayal, 50, 50, 500, 500)

# server = ModularServer(PSOModel,
#                        [grid],
#                        "PSO Model",
#                        model_params())


ImportError: cannot import name 'UserSettableParameter' from 'mesa' (/opt/anaconda3/lib/python3.11/site-packages/mesa/__init__.py)

In [13]:
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import CanvasGrid

# Define model parameters directly
model_params = {
    "N": 100,  # Number of Robots
    "width": 50,
    "height": 50
}

# Define CanvasGrid with hardcoded values
grid = CanvasGrid(agent_portrayal, 50, 50, 500, 500)

# Create ModularServer with hardcoded parameters
server = ModularServer(PSOModel,
                       [grid],
                       "PSO Model",
                       model_params)



In [15]:
if __name__ == '__main__':
    server.launch()


Interface starting at http://127.0.0.1:8521


OSError: [Errno 48] Address already in use

Socket opened!
{"type":"reset"}
